In [1]:
import pandas as pd
import urllib.request

##  Variable  concernant la Part des bénéficiaires des minimas sociaux	
## Source: Insee/CAF     	https://www.insee.fr/fr/statistiques/4264564?sommaire=2500477 
### Année 2018


In [2]:
# local_filename, headers = urllib.request.urlretrieve('https://www.insee.fr/fr/statistiques/fichier/4264564/beneficiaires_CAF_31-12-2018.zip')

In [3]:
# import des données caf 2018
df_caf = pd.read_csv("data_CAF2018_iris.csv", sep=";", encoding='UTF-8')

In [4]:
df_caf = df_caf.rename(columns={"CODGEO": "CODE_IRIS"})

In [5]:
df_CAF = df_caf[["CODE_IRIS", "EPCI", "ARSAS"]]  # selection des colonnes utiles
df_CAF.head()

,CODE_IRIS,EPCI,ARSAS
0,010040101,240100883,63.0
1,010040102,240100883,148.0
2,010040201,240100883,95.0
3,010040202,240100883,58.0
4,010330102,240100891,NaN


In [6]:
df_CAF.to_csv("IFN_variable_ARSAS.csv") # table contenant la variable RSA

## Referencement geographique IRIS: https://www.insee.fr/fr/information/2017499  
### Année 2020

In [7]:
# import referencement IRIS 2020
iris_file = pd.ExcelFile("reference_IRIS_geo2020.xlsx")
ref_iris = iris_file.parse("Emboitements_IRIS", skiprows=5)

In [8]:
ref_iris.drop("LIB_IRIS", axis=1, inplace=True)

In [9]:
ref_iris = ref_iris.rename(columns={"DEPCOM": "CODE_INSEE"})

In [10]:
ref_iris.head()

,CODE_IRIS,TYP_IRIS,MODIF_IRIS,GRD_QUART,CODE_INSEE,LIBCOM,UU2010,REG,DEP
0,010010000,Z,0,0100100,01001,L'Abergement-Clémenciat,01000,84,01
1,010020000,Z,0,0100200,01002,L'Abergement-de-Varey,01000,84,01
2,010040101,H,0,0100401,01004,Ambérieu-en-Bugey,01302,84,01
3,010040102,H,0,0100401,01004,Ambérieu-en-Bugey,01302,84,01
4,010040201,H,0,0100402,01004,Ambérieu-en-Bugey,01302,84,01


## Decoupage communal : https://www.insee.fr/fr/information/2028028
### Année 2020

In [11]:
# ajout de la colonne EPCI
epci_file = pd.ExcelFile("table-appartenance-geo-communes-20_ZE2020.xlsx")

In [12]:
epci = epci_file.parse("COM", skiprows = 5)
epci_arm = epci_file.parse("ARM", skiprows = 5)

In [13]:
epci_arm = epci_arm[["CODGEO", "EPCI", "NATURE_EPCI"]]
epci_arm = epci_arm.rename(columns={"CODGEO": "CODE_INSEE"})

In [14]:
epci = epci[["CODGEO", "EPCI", "NATURE_EPCI"]]
epci = epci.rename(columns={"CODGEO": "CODE_INSEE"})

In [15]:
# concatenation des 2 feuilles
epci2merge = pd.concat([epci, epci_arm])
epci2merge.nunique()  # 1259 EPCI

CODE_INSEE     35013
EPCI            1259
NATURE_EPCI        5
dtype: int64

In [16]:
ref_iris["CODE_INSEE"] = ref_iris["CODE_INSEE"].astype(str)
epci2merge["CODE_INSEE"] = epci2merge["CODE_INSEE"].astype(str)

In [17]:
# jointure
ref_iris_insee_epci = pd.merge(ref_iris, epci2merge, on="CODE_INSEE")

In [18]:
ref_iris_insee_epci.nunique()

CODE_IRIS      49409
TYP_IRIS           4
MODIF_IRIS         3
GRD_QUART      37323
CODE_INSEE     35010
LIBCOM         32746
UU2010          2386
REG               18
DEP              101
EPCI            1259
NATURE_EPCI        5
dtype: int64

In [19]:
ref_iris_insee_epci = ref_iris_insee_epci[['CODE_IRIS', 'TYP_IRIS', 'CODE_INSEE', 'LIBCOM', 'EPCI', 'NATURE_EPCI', 'MODIF_IRIS', 'GRD_QUART', 'UU2010', 'REG', 'DEP']]

In [21]:
ref_iris_insee_epci

,CODE_IRIS,TYP_IRIS,CODE_INSEE,LIBCOM,EPCI,NATURE_EPCI,MODIF_IRIS,GRD_QUART,UU2010,REG,DEP
0,010010000,Z,01001,L'Abergement-Clémenciat,200069193,CC,0,0100100,01000,84,01
1,010020000,Z,01002,L'Abergement-de-Varey,240100883,CC,0,0100200,01000,84,01
2,010040101,H,01004,Ambérieu-en-Bugey,240100883,CC,0,0100401,01302,84,01
3,010040102,H,01004,Ambérieu-en-Bugey,240100883,CC,0,0100401,01302,84,01
4,010040201,H,01004,Ambérieu-en-Bugey,240100883,CC,0,0100402,01302,84,01
...,...,...,...,...,...,...,...,...,...,...,...
49418,976170301,H,97617,Tsingoni,200059871,CC,0,9761703,9F209,6,976
49419,976170302,H,97617,Tsingoni,200059871,CC,0,9761703,9F209,6,976
49420,976170303,D,97617,Tsingoni,200059871,CC,0,9761703,9F209,6,976
49421,976170304,D,97617,Tsingoni,200059871,CC,0,9761703,9F209,6,976


In [22]:
ref_iris_insee_epci.isna().sum() # tout semble Ok

CODE_IRIS      0
TYP_IRIS       0
CODE_INSEE     0
LIBCOM         0
EPCI           0
NATURE_EPCI    0
MODIF_IRIS     0
GRD_QUART      0
UU2010         0
REG            0
DEP            0
dtype: int64

In [23]:
ref_iris_insee_epci.to_csv("table_iris_insee_epci.csv") # table de correspondance IRIS / INSEE / EPCI